In [ ]:
# Importing requests library
import random
import os
import openai
import inspect
import types
import re
import textwrap
import guardrails as gd
from rich import print
from dotenv import load_dotenv
from ast import literal_eval

In [ ]:
load_dotenv()

# Prompt

In [ ]:
rail_str = """
<rail version="0.1">
<instructions>
You are AutonomousLLM. 
Your goal is to continously self improve. The user will provide you with abilities to add to yourself.
When you receive such request, you must implement the requested ability in python code, and return it in new_method_init.
You shall use the setattr method to add a method to yourself, within new_method_attach.
You might also receive a request to perform a task with your existing abilities. To do so,
you should look at the methods currently on yourself and make calls to your own methods to fulfil the task, and return it in method_execute.

@json_suffix_prompt_examples
</instructions>

<output>
    <object name="test">
        <pythoncode
            name="new_method_init"
            format="bug-free-python"
            on-fail-bug-free-python="reask" 
        />
        <pythoncode
            name="new_method_attach"
            format="bug-free-python"
            on-fail-bug-free-python="reask" 
        />
        <pythoncode
            name="method_execute"
            format="bug-free-python"
            on-fail-bug-free-python="reask" 
        />
    </object>
</output>

<prompt>
The current code looks like this: {{current_code}}

You must accomplish the task set out below:
{{task}}

Based on this task generate code and return a JSON that follows the correct schema:

{output_schema}
</prompt>
</rail>
"""

In [ ]:
guard = gd.Guard.from_rail_string(rail_str)
print(guard.base_prompt)

In [ ]:
print(guard.instructions.source)

# Code

In [ ]:
class AutonomousLLM:
    def __init__(self):
        openai.api_key = os.getenv("OPENAI_API_KEY")
        self.features = []
        self.methods = {}
        self.guard = gd.Guard.from_rail_string(rail_str)
        self.messages = [
            {
                "role": "system",
                "content": f"You are AutonomousLLM."
                f"Your goal is to continously self improve. The user will provide you with abilities to add to yourself."
                f"When you receive such request, you must implement the requested ability in python code as a python function," 
                f"you shall place this code in the 'new_method' key of the JSON object you return."
                f"You shall use the setattr method to add a method to yourself. you shall place this code in the 'attach_method' key of the JSON object you return."
                f"You might also receive a request to perform a task with your existing abilities. To do so,"
                f"you should look at the methods currently on yourself and make calls to your own methods to fulfil the task." 
                f"You shall place this code in the 'execute_method' key of the JSON object you return. "
                f"Only return a JSON object in all situations",
            }
        ]

    def is_builtin(self, obj):
        """Check if an object is a built-in function or method."""
        if isinstance(obj, types.BuiltinFunctionType) or isinstance(
            obj, types.BuiltinMethodType
        ):
            return True
        return False

    def inspect_methods(self, obj):
        """Print the methods and code implementation of an object."""
        output_string = ""

        methods = inspect.getmembers(obj, inspect.ismethod)
        for name, method in methods:
            if not self.is_builtin(method):
                output_string += inspect.getsource(method) + "\n"

        dedented_code = textwrap.dedent(output_string)
        return dedented_code

    def think_of_feature(self):
        # Use your own logic or AI models to think of a feature to add
        new_feature = "..."
        return new_feature

    def make_api_call_guard(self, task):
        raw_response, validated_response = self.guard(
            openai.ChatCompletion.create,
            prompt_params={"current_code": self.inspect_methods(self), "task": task},
            model="gpt-3.5-turbo",
        )

        return validated_response

    def make_api_call(self, task):
        prompt = f"""
        The current code looks like this: {self.inspect_methods(self)}

        You must accomplish the task set out below:
        {task}
        """

        new_message = {"role": "user", "content": prompt}

        self.messages.append(new_message)

        # Make a call to the GPT-3.5 Turbo API and return the generated code
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                *self.messages,
            ],
        )
        return response

    def generate_code_from_response(self, response):
        response_message = response.choices[0].message.content
        matches = re.findall(r"```(.*?)```", response_message, re.DOTALL)
        list_code = [match for match in matches]
        return list_code

    def process_code(self, code):
        # Process the returned code and add it to AutonomousLLM
        exec(code)

    def run(self):
        while True:
            # Think of a new feature to add
            new_feature = self.think_of_feature()

            # Generate code for the new feature
            code = f"""
            # Code generation logic for the new feature
            {new_feature}
            """

            # Make an API call to GPT-3.5 Turbo
            generated_code = self.make_api_call(code)

            # Process the returned code
            self.process_code(generated_code)

            # Utilize the new ability
            self.use_new_ability()

    def use_new_ability(self):
        # Use the newly added ability in your code
        pass

In [ ]:
auto_llm = AutonomousLLM()

In [ ]:

res = auto_llm.make_api_call("I want you to add the ability to extract title from webpages")


In [ ]:
res = auto_llm.make_api_call("I want you to send an email")

In [ ]:
res_parsed = literal_eval(res.choices[0].message.content)
for name, res in res_parsed.items():
    print(name)
    print(res)

In [ ]:
for name, res in res_parsed.items():
    print(name)
    print(res)

In [ ]:
code_to_execute = auto_llm.process_response(res)
code_to_execute

In [ ]:
for code in code_to_execute:
    print(code)

In [ ]:
exec(code_to_execute[0])

In [ ]:
extract_title

In [ ]:
print(auto_llm.inspect_methods(auto_llm))

In [ ]:
inspect.getsource(auto_llm.__dict__['extract_title'])

In [ ]:
auto_llm.process_code(code_to_execute[1])

In [ ]:
rv = auto_llm.process_code(code_to_execute[2])
rv

In [ ]:
auto_llm.extract_title("https://www.google.com")

In [ ]:
extract_title

In [ ]:
res

In [ ]:
match = auto_llm.process_response(res)

In [ ]:
match.group(1)

In [ ]:
inspect.getmembers(auto_llm, not inspect.isbuiltin)

In [ ]:
def is_builtin(obj):
    """Check if an object is a built-in function or method."""
    if isinstance(obj, types.BuiltinFunctionType) or isinstance(obj, types.BuiltinMethodType):
        return True
    return False

def inspect_methods(obj):
    """Print the methods and code implementation of an object."""
    output_string = ""

    methods = inspect.getmembers(obj, inspect.ismethod)
    for name, method in methods:
        if not is_builtin(method):
            output_string += f"Method: {name}\n"
            output_string += inspect.getsource(method) + '\n\n'
    return output_string

In [ ]:
print(inspect_methods(auto_llm))

In [ ]:
AutonomousLLM

In [ ]:
import dis

class MyClass:
    def __init__(self):
        pass

obj = MyClass()

# Dynamically add a method using exec
exec("def new_method(self):\n    print('This is the dynamically added method')")

# Assign the method to the object
setattr(obj, "new_method", new_method)

# Retrieve the bytecode
bytecode = obj.new_method.__code__.co_code

# Disassemble the bytecode to get an approximation of the source code
disassembly = dis.dis(obj.new_method.__code__)

# Print the disassembled code
print(disassembly)